In [1]:
import cv2
import numpy as np
import matplotlib.pyplot as plt
from skimage import io, transform
from skimage import filters
from skimage import color


In [ ]:

# Variáveis globais para a interação com o mouse
drawing = False  # True se o mouse está sendo pressionado
ix, iy = -1, -1
mask = None

def calculate_energy(image):
    # Converte a imagem para escala de cinza
    gray_image = color.rgb2gray(image)
    # Calcula o gradiente da imagem
    energy = filters.sobel(gray_image)
    return np.abs(energy)

# Funções para seam carving vertical
def find_vertical_seam(energy):
    r, c = energy.shape
    M = energy.copy()
    backtrack = np.zeros_like(M, dtype=int)

    # Preenchendo a matriz de energia acumulada de cima para baixo
    for i in range(1, r):
        for j in range(c):
            # Trata as bordas separadamente
            if j == 0:
                idx = np.argmin(M[i-1, j:j+2])
                min_energy = M[i-1, idx + j]
                backtrack[i, j] = idx + j
            elif j == c - 1:
                idx = np.argmin(M[i-1, j-1:j+1])
                min_energy = M[i-1, idx + j - 1]
                backtrack[i, j] = idx + j - 1
            else:
                idx = np.argmin(M[i-1, j-1:j+2])
                min_energy = M[i-1, idx + j - 1]
                backtrack[i, j] = idx + j - 1
            M[i, j] += min_energy

    return M, backtrack

def remove_vertical_seam(image, M, backtrack):
    r, c, _ = image.shape
    output = np.zeros((r, c - 1, 3), dtype=image.dtype)
    seam_mask = np.ones((r, c), dtype=bool)
    j = np.argmin(M[-1])  # Inicia a partir do pixel com menor energia acumulada na última linha
    for i in reversed(range(r)):
        seam_mask[i, j] = False  # Marca o pixel da costura a ser removida
        output[i] = np.delete(image[i], j, axis=0)
        j = backtrack[i, j]
    return output, seam_mask

# Funções para seam carving horizontal
def find_horizontal_seam(energy):
    r, c = energy.shape
    M = energy.copy()
    backtrack = np.zeros_like(M, dtype=int)

    # Preenchendo a matriz de energia acumulada da esquerda para a direita
    for j in range(1, c):
        for i in range(r):
            # Trata as bordas separadamente
            if i == 0:
                idx = np.argmin(M[i:i+2, j-1])
                min_energy = M[idx + i, j-1]
                backtrack[i, j] = idx + i
            elif i == r - 1:
                idx = np.argmin(M[i-1:i+1, j-1])
                min_energy = M[idx + i - 1, j-1]
                backtrack[i, j] = idx + i - 1
            else:
                idx = np.argmin(M[i-1:i+2, j-1])
                min_energy = M[idx + i - 1, j-1]
                backtrack[i, j] = idx + i - 1
            M[i, j] += min_energy

    return M, backtrack

def remove_horizontal_seam(image, M, backtrack):
    r, c, _ = image.shape
    output = np.zeros((r - 1, c, 3), dtype=image.dtype)
    seam_mask = np.ones((r, c), dtype=bool)
    i = np.argmin(M[:, -1])  # Inicia a partir do pixel com menor energia acumulada na última coluna
    for j in reversed(range(c)):
        seam_mask[i, j] = False  # Marca o pixel da costura a ser removida
        output[:, j] = np.delete(image[:, j], i, axis=0)
        i = backtrack[i, j]
    return output, seam_mask

# Função para aplicar o seam carving com energia modificada
def seam_carving(image, num_seams_vertical=0, num_seams_horizontal=0, energy_mask=None):
    img = image.copy()

    # Remoção de costuras verticais
    for _ in range(num_seams_vertical):
        energy = calculate_energy(img)
        # Modifica a energia usando a máscara fornecida
        if energy_mask is not None:
            energy += energy_mask
        M, backtrack = find_vertical_seam(energy)
        img, _ = remove_vertical_seam(img, M, backtrack)

    # Remoção de costuras horizontais
    for _ in range(num_seams_horizontal):
        energy = calculate_energy(img)
        # Modifica a energia usando a máscara fornecida
        if energy_mask is not None:
            energy += energy_mask
        M, backtrack = find_horizontal_seam(energy)
        img, _ = remove_horizontal_seam(img, M, backtrack)

    return img

# Função de callback para eventos do mouse
def draw_circle(event, x, y, flags, param):
    global drawing, ix, iy, mask

    if event == cv2.EVENT_LBUTTONDOWN:
        drawing = True
        ix, iy = x, y

    elif event == cv2.EVENT_MOUSEMOVE:
        if drawing:
            cv2.circle(img_display, (x, y), 5, (0, 0, 255), -1)
            cv2.circle(mask, (x, y), 5, 255, -1)

    elif event == cv2.EVENT_LBUTTONUP:
        drawing = False
        cv2.circle(img_display, (x, y), 5, (0, 0, 255), -1)
        cv2.circle(mask, (x, y), 5, 255, -1)

# Código principal
if __name__ == "__main__":
    # Carregar a imagem
    img = io.imread('/home/larissa/Desktop/UFC/Visão/Labs/images/flor.jpg')  # Substitua pelo caminho da sua imagem

    # Reduzir a resolução da imagem (opcional)
    img_resized = transform.resize(img, (img.shape[0] // 2, img.shape[1] // 2), anti_aliasing=True)
    img_resized = (img_resized * 255).astype(np.uint8)

    # Criar uma cópia para exibição e uma máscara
    img_display = img_resized.copy()
    mask = np.zeros(img_resized.shape[:2], dtype=np.uint8)

    # Criar uma janela e definir o callback do mouse
    cv2.namedWindow('Selecione as áreas a serem removidas')
    cv2.setMouseCallback('Selecione as áreas a serem removidas', draw_circle)

    print("Desenhe sobre a imagem as áreas que deseja remover. Pressione 'ESC' para continuar.")

    while True:
        cv2.imshow('Selecione as áreas a serem removidas', img_display)
        k = cv2.waitKey(1) & 0xFF
        if k == 27:  # Tecla 'ESC' para sair
            break

    cv2.destroyAllWindows()

    # Exibir a máscara das áreas selecionadas
    plt.figure(figsize=(10, 5))
    plt.subplot(1, 2, 1)
    plt.imshow(img_resized)
    plt.title('Imagem Original')
    plt.axis('off')

    plt.subplot(1, 2, 2)
    plt.imshow(mask, cmap='gray')
    plt.title('Máscara das Áreas Selecionadas')
    plt.axis('off')
    plt.show()

    # Criar a energia negativa nas áreas selecionadas
    # Aqui, subtraímos um valor grande nas áreas selecionadas para torná-las de energia negativa
    energy_mask = np.zeros(mask.shape, dtype=float)
    energy_mask[mask == 255] = -1000.0  # Valor negativo grande

    # Definir o número de costuras a serem removidas
    num_seams_vertical = 30   # Número de costuras verticais
    num_seams_horizontal = 0  # Número de costuras horizontais

    # Aplicar o seam carving com a energia modificada
    new_image = seam_carving(img_resized, num_seams_vertical, num_seams_horizontal, energy_mask=energy_mask)

    # Exibir a imagem resultante
    plt.figure(figsize=(15, 5))
    plt.subplot(1, 3, 1)
    plt.imshow(img_resized)
    plt.title('Imagem Original')
    plt.axis('off')

    plt.subplot(1, 3, 2)
    plt.imshow(mask, cmap='gray')
    plt.title('Máscara das Áreas Selecionadas')
    plt.axis('off')

    plt.subplot(1, 3, 3)
    plt.imshow(new_image)
    plt.title('Imagem após Seam Carving')
    plt.axis('off')

    plt.tight_layout()
    plt.show()
